# BIDS Bro Inspection

In [1]:
import glob, os
import pandas as pd
import pdb

## BIDS directory on renci
*make script to automize gathering of subject count, etc...*  
**The directory-**  
`$ pwd`  
`/projects/niblab/bids_projects/Experiments/bro/bids`    
**Number of subject directories on RENCI-**    
`$ ls -d sub-*/ses-1 | wc -l` 
`43`    
`$ ls -d sub-*/ses-2 | wc -l` `50`  

In [2]:
sessions=['ses-1', 'ses-2']

## Volume Check  
Script to produce volumes file(run on renci): [`bids_quality_check.py`](https://github.com/niblunc/BRO/blob/master/BIDS/qa/bids_quality_check.py)   
*add functionality here to produce text file automatically*   
See [`CHANGELOG`](https://github.com/niblunc/BRO/blob/master/CHANGELOG.md) for further details on volume check and corrections status.


In [3]:
print("Files found not matching expected volume: \n{ pe: 193, training: 243, resting: 147 }")
  
filename='/Users/nikkibytes/Documents/niblunc/bro/bids/qa/bad_volumes.txt'
vol_dict = {}
with open(filename) as f:
    content=f.readlines()
    for line in content:
        if "training" in line:
            file=line.split("\t")[0]
            vol=line.split("\t")[1].strip()
        else:
            file=line.split("\t\t")[0]
            vol=line.split("\t\t")[1].strip()
        temp_tuple=(file,vol)
        sub_id = file.split(":")[1].split("_")[0]
        if sub_id not in vol_dict:
            vol_dict[sub_id] = [temp_tuple]
        else:
            vol_dict[sub_id].append(temp_tuple)

for sub in vol_dict:
    print("\nID: {}\n".format(sub))
    for x in vol_dict[sub]:
        if "training" in x[0]:
            print("{} \t{} ".format(x[0], x[1]))
        else:
            print("{} \t\t{} ".format(x[0], x[1]))
        print("********************************************************************")

Files found not matching expected volume: 
{ pe: 193, training: 243, resting: 147 }

ID: sub-006

File:sub-006_ses-1_task-resting_bold.nii.gz  		Volume:93 
********************************************************************
File:sub-006_ses-1_task-training_run-1_bold.nii.gz  	Volume:182 
********************************************************************
File:sub-006_ses-1_task-training_run-2_bold.nii.gz  	Volume:185 
********************************************************************
File:sub-006_ses-1_task-pe_run-1_bold.nii.gz  		Volume:141 
********************************************************************
File:sub-006_ses-1_task-pe_run-2_bold.nii.gz  		Volume:141 
********************************************************************

ID: sub-020

File:sub-020_ses-1_task-pe_run-1_bold.nii.gz  		Volume:50 
********************************************************************

ID: sub-031

File:sub-031_ses-1_task-training_run-1_bold.nii.gz  	Volume:228 
****************************

## Missing files in subjects by session

In [ ]:
# run this on renci to get output


ses1_dcms=[x.split("/")[-1] for x in glob.glob('/projects/niblab/bids_projects/Experiments/bro/bids/sourcedata/DICOM/ses-1/sub-*')]
ses2_dcms=[x.split("/")[-1] for x in glob.glob('/projects/niblab/bids_projects/Experiments/bro/bids/sourcedata/DICOM/ses-2/sub-*')]

ses1_dcms_ct= len(dcms_ses1)
ses2_dcms_ct= len(dcms_ses2)

ses1_bids=[x.split("/")[-2] for x in glob.glob('/projects/niblab/bids_projects/Experiments/bro/bids/sub-*/ses-1')]
ses2_bids=[x.split("/")[-2] for x in glob.glob('/projects/niblab/bids_projects/Experiments/bro/bids/sub-*/ses-2')]

ses1_bids_ct= len(ses1_bids)
ses2_bids_ct= len(ses2_bids)


ses1_mia_lst=set(ses1_dcms) - set(ses1_bids)
ses2_mia_lst=set(ses2_dcms) - set(ses2_bids)

if not ses1_mia_lst:
    ses1_mia_lst="None missing"
if not ses2_mia_lst:
    ses2_mia_lst="None missing"
    
print("> Session 1: \nDICOM subject directory count: \t{} \
      \nBIDS subject directory count: \t{} \nSubjects missing from BIDS directory: \t{} \n \
      \n\n> Session 2: \nDICOM subject directory count: \t{} \
      \nBIDS subject directory count: \t{} \
      \nSubjects missing from BIDS directory: \t{} \n".format(ses1_dcms_ct,ses1_bids_ct,ses1_mia_lst,ses2_dcms_ct,ses2_bids_ct, ses2_mia_lst))

`> Session 1:`    
`DICOM subject directory count: 	50 `        
`BIDS subject directory count: 	 43`  
`Subjects missing from BIDS directory: 	{'sub-054', 'sub-013', 'sub-017', 'sub-035', 'sub-003', 'sub-004', 'sub-018'} `       

`> Session 2:`  
`DICOM subject directory count: 	50`       
`BIDS subject directory count: 	 50 `  
`Subjects missing from BIDS directory: 	None missing `  

In [4]:
from IPython.core import display as ICD

for sess_id in sessions:
    if "1" in sess_id:
        print("\n> SESSION 1\n")
    else:
        print("\n> SESSION 2\n")
        
    ses_mia=pd.read_csv('bro_{}_missing.csv'.format(sess_id),sep="\t")
    ses_mia.set_index("Unnamed: 0", inplace=True)
    ses_mia.index.names=["subjects"]
    
    ses_pmia=pd.read_csv('bro_{}_partial_missing.csv'.format(sess_id),sep="\t")
    ses_pmia.set_index("Unnamed: 0", inplace=True)
    ses_pmia.index.names=["subjects"]
    
    ses_fd=pd.read_csv('bro_{}_found.csv'.format(sess_id),sep="\t")
    ses_fd.set_index("Unnamed: 0", inplace=True)
    ses_fd.index.names=["subjects"]
    
    

    print("All files found missing: \t{}\n".format(' '.join(ses_mia.index.values)))
    print("Some files found missing: \t{}\n".format(' '.join(ses_pmia.index.values)))
    print("All files found: \t\t{}\n".format(' '.join(ses_fd.index.values)))

    print("\nDataframe (Missing files): ")
    ICD.display(ses_mia)
    print("\nDataframe (Partial files found): ")
    ICD.display(ses_pmia)
    print("\nDataframe (All files found): ")
    ICD.display(ses_fd)
    
    


> SESSION 1

All files found missing: 	sub-001 sub-002 sub-003 sub-004 sub-013 sub-017 sub-018 sub-027 sub-042 sub-054

Some files found missing: 	sub-010 sub-012 sub-037 sub-040 sub-045 sub-052

All files found: 		sub-006 sub-007 sub-008 sub-009 sub-011 sub-014 sub-015 sub-016 sub-019 sub-020 sub-021 sub-022 sub-023 sub-025 sub-026 sub-028 sub-029 sub-030 sub-031 sub-032 sub-033 sub-034 sub-036 sub-038 sub-039 sub-041 sub-043 sub-044 sub-046 sub-047 sub-048 sub-049 sub-050 sub-051 sub-053 sub-055 sub-056


Dataframe (Missing files): 


,anat,fmap,func,runs_pe,runs_rest,runs_train
subjects,,,,,,
sub-001,0,0,0,0,0,0
sub-002,0,0,0,0,0,0
sub-003,0,0,0,0,0,0
sub-004,0,0,0,0,0,0
sub-013,0,0,0,0,0,0
sub-017,0,0,0,0,0,0
sub-018,0,0,0,0,0,0
sub-027,0,0,0,0,0,0
sub-042,0,0,0,0,0,0



Dataframe (Partial files found): 


,anat,fmap,func,runs_pe,runs_rest,runs_train
subjects,,,,,,
sub-010,1,0,0,0,0,0
sub-012,1,0,0,0,0,0
sub-037,1,0,0,0,0,0
sub-040,1,0,0,0,0,0
sub-045,0,5,5,2,1,2
sub-052,1,0,0,0,0,0



Dataframe (All files found): 


...

,anat,fmap,func,runs_pe,runs_rest,runs_train
subjects,,,,,,
sub-020,0,0,0,0,0,0
sub-037,0,0,0,0,0,0
sub-055,0,0,0,0,0,0



Dataframe (Partial files found): 


,anat,fmap,func,runs_pe,runs_rest,runs_train
subjects,,,,,,
sub-003,0,4,4,2,0,2
sub-008,1,0,0,0,0,0
sub-025,1,0,0,0,0,0
sub-042,0,3,3,2,1,0



Dataframe (All files found): 


,anat,fmap,func,runs_pe,runs_rest,runs_train
subjects,,,,,,
sub-001,1,5,5,2,1,2
sub-002,1,5,5,2,1,2
sub-004,1,5,5,2,1,2
sub-006,1,5,5,2,1,2
sub-007,1,5,5,2,1,2
sub-009,1,6,6,2,1,3
sub-010,1,5,5,2,1,2
sub-011,1,5,5,2,1,2
sub-012,1,5,5,2,1,2
